In [1]:
import numpy
import tensorflow as tf
import os
from tensorflow.keras.models import load_model
import random
import json
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer

stemmer = LancasterStemmer()



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
with open("/content/intents.json") as file:
	data = json.load(file)

In [3]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data["intents"]:
	for pattern in intent["patterns"]:
		wrds = nltk.word_tokenize(pattern)
		words.extend(wrds)
		docs_x.append(wrds)
		docs_y.append(intent["tag"])

	if intent["tag"] not in labels:
		labels.append(intent["tag"])

words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)


training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
	bag = []

	wrds = [stemmer.stem(w) for w in doc]

	for w in words:
		if w in wrds:
			bag.append(1)
		else:
			bag.append(0)
	output_row = out_empty[:]
	output_row[labels.index(docs_y[x])] = 1

	training.append(bag)
	output.append(output_row)

training = numpy.array(training)
output = numpy.array(output)

In [4]:
tf.keras.backend.clear_session()
model_file = "model.h5"

if os.path.exists(model_file):
    model = load_model(model_file)
else:
  model = Sequential([
    Dense(8, input_shape=(len(training[0]),), activation='relu'),
    Dense(8, activation='relu'),
    Dense(len(output[0]), activation='softmax')
])



model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(training, output, epochs=1000, batch_size=8, verbose=1)
model.save(model_file)

Epoch 1/1000
24/24 [==============================] - 2s 3ms/step - loss: 3.7822 - accuracy: 0.0053
Epoch 2/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.7710 - accuracy: 0.0106
Epoch 3/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.7611 - accuracy: 0.0160
Epoch 4/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.7496 - accuracy: 0.0372
Epoch 5/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.7366 - accuracy: 0.0426
Epoch 6/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.7197 - accuracy: 0.0426
Epoch 7/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.6974 - accuracy: 0.0532
Epoch 8/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.6705 - accuracy: 0.0532
Epoch 9/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.6371 - accuracy: 0.0638
Epoch 10/1000
24/24 [==============================] - 0s 3ms/step - loss: 3.5982 - accuracy: 0.0691

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [5]:
def bag_of_words(s,words):
	bag = [0 for _ in range(len(words))]



	s_words = nltk.word_tokenize(s)
	s_words = [stemmer.stem(word.lower()) for word in s_words]


	for se in s_words:
		for i, w in enumerate(words):
			if w == se:
				bag[i] = 1



	return numpy.array(bag).reshape(1, len(words))

In [6]:
def chat():
	print("Start Talking with the bot(type quit to stop!")
	while True:
		inp = input("You: ")
		if inp.lower() == "quit":
			break

		results = model.predict([bag_of_words(inp,words)])[0]
		results_index = numpy.argmax(results)
		tag = labels[results_index]

		if results[results_index] > 0.5:
			for tg in data["intents"]:
				if tg['tag'] == tag:
					responses = tg['responses']
			print(random.choice(responses))
			print("\n")

		else:
			print("I didnt get that, try again")

In [9]:
chat()

Start Talking with the bot(type quit to stop!
You: Sun burn
1/1 [==============================] - 0s 18ms/step
 


You: Sun Burn
1/1 [==============================] - 0s 17ms/step
 


You: quit
